In [1]:
import pandas as pd
import numpy as np

In [2]:
#load training dataset
#ratings_big = pd.read_csv('train_dt/ratings.csv') 
    # movies_metadata와 merge 한 후의 갯수 : 11428646
ratings = pd.read_csv('train_dt/ratings_small.csv')

movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')

credits = pd.read_csv('train_dt/credits.csv')

keywords = pd.read_csv('train_dt/keywords.csv')

links = pd.read_csv('train_dt/links.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_7984\1772567852.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('train_dt/movies_metadata.csv')


ratings preprocessing

In [3]:
#추후 merge를 위해 movieId를 id로 통일
#ratings.rename(columns={'movieId':'id'}, inplace = True)

In [4]:
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
#ratings.loc['id'] = ratings['id'].astype('str')
#ratings['id'] = pd.to_numeric(ratings['id'])

movies_metadata processing

In [5]:
#나중에 최종결과 보여줄 때 movies_metadata_origin을 사용할 예정
movies_metadata_origin = movies_metadata

In [6]:
#계산에 불필요한 ft drop
movies_metadata.drop(['homepage','overview', 'tagline', 'poster_path'], axis = 1, inplace = True)

#이 두 ft는 kaggle에는 안뜨는 데 다운받으면 있음
#의미를 몰라서 일단 drop
movies_metadata.drop(['vote_average','vote_count'], axis = 1, inplace = True)

#movies_metadata_origin[movies_metadata_origin['production_companies'] == "[]"] -> 결과 : 11559 
#null값이 11559개(전체 movie dataset : 45031개)라 총 1/4정도를 차지해서 없애는 것이 낫다고 판단
movies_metadata.drop(['production_companies'], axis = 1, inplace = True)

In [7]:
#belongs_to_collection null 값을 no collection 값으로 변경
movies_metadata['belongs_to_collection'].replace(np.nan,'no collection', inplace = True)

In [8]:
#null값 없애기
movies_metadata.dropna(inplace = True)

In [9]:
#'genres', 'proudction_countries'가 null인 경우 drop
movies_metadata = movies_metadata.loc[movies_metadata_origin['genres'] != "[]"]
movies_metadata = movies_metadata.loc[movies_metadata_origin['production_countries'] != "[]"]

In [10]:
#movies_metadata에서 budget이 0 인 것 개수
len(movies_metadata.loc[movies_metadata['budget'] == '0']['budget'])

29600

In [11]:
movies_metadata['budget'] = pd.to_numeric(movies_metadata['budget'])

In [12]:
#movies_metadata['budget'] 0인 것은 나머지의 평균으로 넣는다.
budget_mean = movies_metadata.loc[movies_metadata['budget'] != 0]['budget'].mean()
movies_metadata['budget'].replace(0, budget_mean, inplace = True)

In [13]:
#'id'형식이 날짜 형식인 경우 -> 잘못된 경우
#ex) ValueError: Unable to parse string "2012-09-29" at position 29502
movies_metadata['isIdRight'] = movies_metadata['id'].str.contains('|'.join('-'))
movies_metadata = movies_metadata[movies_metadata['isIdRight'] == False]
movies_metadata.drop(['isIdRight'], axis = 1, inplace = True)

In [14]:
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
movies_metadata.loc['id'] = movies_metadata['id'].astype('str')
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'])

credits preprocessing

In [18]:
#credits의 [] - null값 제거
credits = credits.loc[credits['cast'] != "[]"]
credits = credits.loc[credits['crew'] != "[]"]

In [19]:
#추후 merge를 위해 index id의 값들을 숫자형태로 변경
#credits['id'] = pd.to_numeric(credits['id'])

keywords preprocessing

In [20]:
keywords = keywords.loc[keywords['keywords'] != "[]"]

link preprocessing

In [21]:
links = links.dropna()#45843 -> 45624

merging for making complete training dataset

In [22]:
all_movie_df = pd.merge(movies_metadata, credits)

In [23]:
all_movie_df#null값 없음

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",3.000000e+07,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,tt0114709,en,Toy Story,21.946943,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,False,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,no collection,6.500000e+07,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,tt0113497,en,Jumanji,17.015539,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,False,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",2.232716e+07,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,tt0113228,en,Grumpier Old Men,11.7129,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,False,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,no collection,1.600000e+07,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,tt0114885,en,Waiting to Exhale,3.859495,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,False,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",2.232716e+07,"[{'id': 35, 'name': 'Comedy'}]",11862.0,tt0113041,en,Father of the Bride Part II,8.387519,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,False,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37232,False,no collection,2.232716e+07,"[{'id': 27, 'name': 'Horror'}]",289923.0,tt0252966,en,The Burkittsville 7,0.38645,"[{'iso_3166_1': 'US', 'name': 'United States o...",2000-10-03,0.0,30.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Burkittsville 7,False,"[{'cast_id': 2, 'character': 'Branwall', 'cred...","[{'credit_id': '5403d669c3a3682d9800427d', 'de..."
37233,False,no collection,2.232716e+07,"[{'id': 878, 'name': 'Science Fiction'}]",222848.0,tt0112613,en,Caged Heat 3000,0.661558,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Caged Heat 3000,False,"[{'cast_id': 1, 'character': 'Kira (as Cassand...","[{'credit_id': '5757f36ac3a3687d6f000e8a', 'de..."
37234,False,no collection,2.232716e+07,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",30840.0,tt0102797,en,Robin Hood,5.683753,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1991-05-13,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Robin Hood,False,"[{'cast_id': 1, 'character': 'Sir Robert Hode'...","[{'credit_id': '52fe44439251416c9100a899', 'de..."
37235,False,no collection,2.232716e+07,"[{'id': 18, 'name': 'Drama'}]",111109.0,tt2028550,tl,Siglo ng Pagluluwal,0.178241,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,Century of Birthing,False,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."


In [24]:
all_movie_df = pd.merge(all_movie_df, ratings)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [ ]:
all_movie_df.isnull().sum()

In [ ]:
#rating 별로 movie df가 합쳐진 것을 알 수 있음
print(len(ratings[ratings['id'] == 949]))
print(len(all_movie_df[all_movie_df['id'] == 949]))

In [ ]:
#추후 merge를 위해 index id의 값들을 문자로 변경
#ratings.loc['movieId'] = ratings['movieId'].astype('str')
ratings['movieId'] = pd.to_numeric(ratings['movieId'])

#추후 merge를 위해 index id의 값들을 문자로 변경
#movies_metadata.loc['id'] = movies_metadata['id'].astype('str')
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'])

user 기반 filtering - matrix-factorization
https://www.kaggle.com/code/abhinavjhanwar/movierecommender-collaborative-lightfm

In [26]:
data = pd.merge(ratings, movies_metadata[['id', 'title']], left_on='movieId', right_on='id')
print(len(ratings))
#count 열 추가
# get total counts of no. of occurence of movie
data['count'] = data.groupby('movieId').transform('count')['userId']

# fetch top 100 movies based on count
movieId = data.drop_duplicates('movieId').sort_values(
    'count', ascending=False).iloc[:100]['movieId']

# filter out data as per the movieId
data = data[data['movieId'].isin(movieId)].reset_index(drop=True)

# get total counts of movies each user has seen
data['count'] = data.groupby('userId').transform('count')['movieId']

# fetch top 20000 users based on no. of movies watched
userId = data.drop_duplicates('userId').sort_values(
    'count', ascending=False).iloc[:20001]['userId']

# filter out data as per the userId
data = data[data['userId'].isin(userId)].reset_index(drop=True)

# create a user movie rating matrix
df = data.pivot(index='userId', columns='movieId', values='rating')
df

100004


movieId,2,6,19,21,25,95,104,110,111,141,...,3578,4011,4226,4886,4896,4973,4993,4995,8961,58559
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,4.0,NaN,3.0,2.0,3.0,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
# replace NaN with user based average rating
df_imputed = df.fillna(df.mean(axis=0))

# get similarity between all users
similarity_matrix = cosine_similarity(df_imputed.values)

In [31]:
def get_recommendation_collaborative(user_index):
    idx = user_index
    sim_scores = list(enumerate(similarity_matrix[idx]))
    
    # get movies that are unrated by the given user
    unrated_movies = df.iloc[idx][df.iloc[idx].isna()].index
    print(unrated_movies)
    # get weighted ratings of unrated movies by all other users
    movie_ratings = (df_imputed.iloc[similarity_matrix[idx]][unrated_movies].T * [
        x[1] for x in sim_scores]).T

    # get top 100 similar users by skipping the current user
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:101]

    # get mean of movie rating by top 100 most similar users for the unrated movies
    movie_ratings = movie_ratings.iloc[[x[0] for x in sim_scores]].mean()

    # get recommended movie titles in sorted order
    recommended_movies = movies_metadata[movies_metadata['id'].isin(movie_ratings.reset_index().sort_values(
        0, ascending=False)['movieId'])][['title', 'id']]
    assumed_ratings = sorted(movie_ratings, reverse=True)

    return pd.DataFrame({'movieId':recommended_movies[:10]['id'], 
                         'Recommended Movie':recommended_movies[:10]['title'], 
                         'Assumed Rating':assumed_ratings[:10]})
user_id = 1
recommended_movies = get_recommendation_collaborative(user_id).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==df.index[user_id]].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
print(temp)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

Int64Index([   2,    6,   19,   21,   25,   95,  104,  111,  141,  150,  153,
             165,  185,  223,  231,  253,  260,  288,  292,  293,  300,  316,
             329,  339,  344,  349,  364,  380,  434,  440,  442,  454,  457,
             480,  500,  539,  541,  551,  586,  587,  590,  597,  608,  648,
             750,  780,  858,  912,  919,  924, 1073, 1089, 1213, 1246, 1259,
            1265, 1387, 1393, 1584, 1610, 1682, 1732, 1917, 1923, 1961, 1968,
            2000, 2011, 2012, 2115, 2324, 2355, 2502, 2791, 3114, 3578, 4011,
            4226, 4886, 4896, 4973, 4993, 4995, 8961],
           dtype='int64', name='movieId')
   rating                               title  userId
0     5.0            The Million Dollar Hotel       3
1     5.0                      License to Wed       3
2     4.5  Terminator 3: Rise of the Machines       3
3     4.5                    All the Way Boys       3
4     4.0                   Three Colors: Red       3
5     4.0           Monsieur  Hul

,movieId,Recommended Movie,Assumed Rating,userId,Movie Watched,Rated
0,6.0,Judgment Night,4.976996,3,The Million Dollar Hotel,5.0
1,329.0,Jurassic Park,4.976996,3,License to Wed,5.0
2,858.0,Sleepless in Seattle,4.976996,3,Terminator 3: Rise of the Machines,4.5
3,223.0,Rebecca,4.466854,3,All the Way Boys,4.5
4,260.0,The 39 Steps,4.237112,3,Three Colors: Red,4.0
5,454.0,Romeo + Juliet,4.215558,3,Monsieur Hulot's Holiday,4.0
6,500.0,Reservoir Dogs,4.207824,3,Say Anything...,4.0
7,185.0,A Clockwork Orange,4.205140,3,Rope,3.5
8,539.0,Psycho,4.202226,3,Young and Innocent,3.5
9,339.0,Night on Earth,4.190157,3,Once Were Warriors,3.0


In [32]:
user_id = 15554
recommended_movies = get_recommendation_collaborative(user_id).reset_index(drop=True)
# get other high rated movies by user
temp = data[data['userId']==df.index[user_id]].sort_values(
    'rating', ascending=False)[['rating', 'title', 'userId']].iloc[:10].reset_index(drop=True)
recommended_movies['userId'] = temp['userId']
recommended_movies['Movie Watched'] = temp['title']
recommended_movies['Rated']= temp['rating']
recommended_movies

IndexError: index 15554 is out of bounds for axis 0 with size 655

In [45]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
reader = Reader()
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [41]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5) #cross-validation으로 교체하면 사용 가능하다고 함

In [46]:
svd = SVD()

#evaluate(svd, data, measures=['RMSE', 'MAE'])#cross-validation으로 교체
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9020  0.8913  0.8993  0.9028  0.8898  0.8971  0.0054  
MAE (testset)     0.6944  0.6832  0.6932  0.6952  0.6862  0.6904  0.0048  
Fit time          0.79    0.92    0.85    0.86    0.77    0.84    0.05    
Test time         0.11    0.12    0.27    0.11    0.17    0.15    0.06    


{'test_rmse': array([0.90204962, 0.89132426, 0.89931306, 0.90283367, 0.88983194]),
 'test_mae': array([0.69444422, 0.68318007, 0.69322803, 0.69516724, 0.68620419]),
 'fit_time': (0.7919204235076904,
  0.9185419082641602,
  0.8526871204376221,
  0.8646471500396729,
  0.7749223709106445),
 'test_time': (0.10569953918457031,
  0.12071037292480469,
  0.26628661155700684,
  0.11273360252380371,
  0.16655683517456055)}

In [55]:
trainset = data.build_full_trainset()
trainset
#svd.train(trainset)
svd.predict(1000000, 302, 3)

Prediction(uid=1000000, iid=302, r_ui=3, est=3.6034595411280024, details={'was_impossible': False})

content 기반 filtering - cosine similarity